### <font color=#ff0000>宏基因组分析——基于reads分析流程</font>

#### 1. 环境配置

#### 1.1 本机环境配置

#### 1.1.1 安装Docker (https://www.runoob.com/docker/centos-docker-install.html)

#### 1.1.2 加载Docker镜像 (Jupyter Notebook + sratoolkit + biobakery-workflows-add-r)

以下两句命令在终端运行，运行之后在浏览器输入：http://localhost:7777/lab?， 进入Jupyter环境执行后续命令。

In [ ]:
!sudo docker load -i ddhmed/lab-python-r-julia.tar

In [ ]:
!sudo docker run -it --privileged=TRUE --rm -p 7777:8888 -v /Volumes/Untitled:/work -v /var/run/docker.sock:/var/run/docker.sock -v $(which docker):/bin/docker ddhmed/lab-python-r-julia:v1 jupyter lab --allow-root

命令主要参数说明：
* i : 交互模式, 允许你对容器内的标准输入 (STDIN) 进行交互
* t : 终端, 在新容器内指定一个伪终端或终端
* p : 端口映射, 将主机端口映射于容器内部端口
* v : 目录挂载, 将主机目录空间映射于容器内部目录空间,/Volumes/Untitled为工作目录，可根据自己本机实际目录修改
* ddhmed/lab-python-r-julia:v1 : 启动的镜像和版本号
* privileged=TRUE ：允许使用本机命令权限
* rm : 退出后该docker自动在后台删除

以下所有命令操作都在Jupyter环境下进行，/Volumes/Untitled为工作目录，可根据自己本机实际目录修改

In [ ]:
!sudo docker load -i pegi3s-sratoolkit-latest.tar

In [2]:
!sudo docker load -i biobakery-workflows-add-r-6.tar

Loaded image: biobakery-workflows-add-r-6:latest


#### 1.2 服务器环境配置 (根据同济服务器配置)

#### 1.2.1 安装biobakery-workflows (http://huttenhower.sph.harvard.edu/biobakery_workflows)

#### 1.2.2 安装MelonnPan (https://huttenhower.sph.harvard.edu/melonnpan)

#### 1.2.3 安装MetAML (http://segatalab.cibio.unitn.it/tools/metaml/)

#### 2. Jupyter、docker和服务器运行说明

#### 2.1 Jupyter中，以！为开头的代码是bash命令，运行之前在上方菜单栏选择Kernel->Change Kernel->Python 3。

#### 2.2 Jupyter中，不以！为开头的代码是R语言命令，运行之前在上方菜单栏选择Kernel->Change Kernel->R。

#### 2.3 Jupyter中，图表显示所在行代码，需要在上方Code改为MarkDown之后运行。

#### 2.4 docker运行过程中，可以在终端界面输入docker stats -a，实时查看docker命令运行占据本机CPU比例和内存比例，尽量保证CPU比例不超过100%，如果CPU占比例太多，则减少正在运行的docker命令，如果内存占比例太多，则增加docker内存，在docker菜单中选择Preferences->Advanced。本流程可以docker为设置3个CPU和2G内存。

#### 2.5 服务器运行过程中，通过qstat -f查看任务所在节点是否负载，负载的节点有a标记，如果负载，则需要调整命令中使用的线程数量。

#### 3. 具体步骤

#### 3.1 测序数据准备

#### 3.1.1 原始数据SRA格式转化为fastq格式

说明：

3.1.1.1 该命令在原始数据为SRA格式时运行，若原始数据格式已经是fastq格式，则不需要运行该命令。

3.1.1.2 在3个CPU的docker环境下，建议每次同时运行2个fasterq-dump命令，每次运行时间大概10-20分钟。

In [1]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'mkdir -p /home/data/results/fasterq-dump && \
        file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[0]} -O /home/data/results/fasterq-dump'

spots read      : 12,230,104
reads read      : 24,460,208
reads written   : 24,460,208


In [21]:
!bash -c 'file=(`ls /work/input_data/sra/*`) && echo ${file[0]}'

/work/input_data/sra/SRR5935846


这里结果显示第一个样本SRR5935846一共有24,460,208条测序短序列reads。

In [2]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[1]} -O /home/data/results/fasterq-dump'

spots read      : 6,776,291
reads read      : 13,552,582
reads written   : 13,552,582


In [3]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[2]} -O /home/data/results/fasterq-dump'

spots read      : 12,216,205
reads read      : 24,432,410
reads written   : 24,432,410


In [4]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[3]} -O /home/data/results/fasterq-dump'

spots read      : 10,685,778
reads read      : 21,371,556
reads written   : 21,371,556


In [5]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[4]} -O /home/data/results/fasterq-dump'

spots read      : 13,821,817
reads read      : 27,643,634
reads written   : 27,643,634


In [6]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[5]} -O /home/data/results/fasterq-dump'

spots read      : 20,893,032
reads read      : 41,786,064
reads written   : 41,786,064


In [7]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[6]} -O /home/data/results/fasterq-dump'

spots read      : 17,696,087
reads read      : 35,392,174
reads written   : 35,392,174


In [27]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[7]} -O /home/data/results/fasterq-dump'

spots read      : 23,274,888
reads read      : 46,549,776
reads written   : 46,549,776


In [28]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[8]} -O /home/data/results/fasterq-dump'

spots read      : 8,735,538
reads read      : 17,471,076
reads written   : 17,471,076


In [29]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[9]} -O /home/data/results/fasterq-dump'

spots read      : 9,913,957
reads read      : 19,827,914
reads written   : 19,827,914


In [30]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'file=(`ls /home/data/input_data/sra/*`) && \
        fasterq-dump --split-3 ${file[10]} -O /home/data/results/fasterq-dump'

spots read      : 12,089,985
reads read      : 24,179,970
reads written   : 24,179,970


#### 3.1.2 修改fastq文件后缀，标记双端的forward/reverse测序序列，以便后续的预处理、群落结构分析和群落功能分析中识别forward/reverse序列。

说明：

3.1.2.1 对于双端数据，同时运行以下两句命令修改。

3.1.2.2 对于单端数据，则不需要修改文件后缀。

In [31]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'rename.ul _1 .R1 /home/data/results/fasterq-dump/*_1.fastq'

In [32]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data pegi3s/sratoolkit:latest bash \
        -c 'rename.ul _2 .R2 /home/data/results/fasterq-dump/*_2.fastq'

#### 3.1.3 修改fastq文件表述行，标记双端的forward/reverse测序序列，以便后续质控算法中识别forward/reverse序列。

说明：

3.1.3.1 对于双端序列，forward序列的描述行以 /1结尾，reverse序列的描述行以 /2结尾。

3.1.3.2 对于单端序列，不需要做此步骤。

3.1.3.3 下面每一条python程序占用CPU比较高，约80%-90%，每个文件命令运行时间约15分钟，一共22个文件，总运行时间约5.5小时。

In [46]:
!sudo docker run --rm -v /Volumes/Untitled/:/home/data biobakery-workflows-add-r-6 bash \
    -c 'file=(`ls /home/data/results/fasterq-dump/*.fastq`) && \
    mkdir -p /home/data/results/fasterq-dump-change-name/ && \
    for x in ${file[*]}; do python /home/data/SeqIO_change_fastq_ID.py $x; done'

#### 3.2 预处理、群落结构丰度谱和功能丰度谱构建

说明：

3.2.1 预处理、群落结构丰度谱和群落功能丰度谱分析属于三个串联步骤，目前有整合命令可以运行，该命令在服务器跑，需要安装biobakery_workflows软件，输入文件为3.1.3步骤处理好的fasterq-dump-change-name文件夹。在后面结果展示中，会将整合命令中每个步骤执行的具体命令一一说明。

3.2.2 服务器上运行命令：
biobakery_workflows wmgx --input fasterq-dump-change-name/ --output biobakery_workflows/ --threads 8 --input-extension fastq --local-jobs 8

3.2.3 该命令中--local-jobs参数表示每次并行运行的样本数量，--threads表示每个运行样本所用线程数，该参数设置需要根据服务器节点数量而定，尽量保证local-jobs*threads<服务器总节点数量，该命令运行时间大概是7个小时，运行完之后，从服务器下载biobakery_workflows文件夹到本机/Volumes/Untitled/results。

3.2.4 运行结果下载到本机之后，用以下命令在/home/data/results/biobakery_workflows_output_vis文件夹中生成结果报告和图表，以下对每一步骤结果及其所用命令进行展示。注意：3.2.5-3.2.7步骤的命令不需运行，这些命令已包含3.2.2总命令中，3.2.5-3.2.7步骤的命令仅作为内部命令的详细讲解。

In [ ]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'biobakery_workflows wmgx_vis --input /home/data/results/biobakery_workflows/ --output /home/data/results/biobakery_workflows_output_vis --project-name IBD_test'

3.2.5 预处理步骤

3.2.5.1 示例命令
* kneaddata –input SRR5935917.R1.fastq –output main –threads 8
–output-prefix SRR5935917 –input SRR5935917.R2.fastq –cat-final-output
–reference-db kneaddata_db_human_genome –serial –run-trf

3.2.5.2 主要参数说明
 * –cat-final-output：最终输出所有fastq文件合并成一个文件
 * –reference-db：宿主参考基因组数据库，默认是以人类作为宿主参考基因组，如果是其它物种，则需要提供其它宿主物种对应的bowtie index数据库；而在biobakery_workflows命令中，可以通过修改对应的参数--qc-options
 * –serial：在bowtie比对步骤中，过滤在多个数据库的串行输入，以便每次在每个数据库中搜索一部分测序序列reads，提高运算速度
 * run-trf：利用TRF软件去除串联重复序列

3.2.5.3 样本质控统计图表

![**wmgx_report_figure2_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure2_1.png)

![**wmgx_report_figure5_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure5_1.png)

图表说明：

* Raw：原始reads数量（双端测序统计一端reads数量）
* Trim：根据质量分数<20和合格序列长度<50%原始序列长度的质控准则，去除低质量碱基后的reads数量（双端测序统计一端reads数量）
* hg38：去除与hg38宿主人类参考基因组比对上的序列，合并双端测序所有序列，统计reads数量

3.2.5.4 样本孤儿序列统计图表

![**wmgx_report_figure3_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure3_1.png)

![**wmgx_report_figure6_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure6_1.png)

图表说明：

hg38 orphan1和hg38 orphan2表示分别在双端的forward和reverse序列中未与hg38宿主人类基因组比对，而其另一端比对上hg38基因组，另一端序列属于宿主污染，而其本身序列并不属于宿主污染。

3.2.6 群落结构丰度谱构建步骤

3.2.6.1 示例命令
* metaphlan2.py SRR5935917.fastq –input_type fastq –output_file
SRR5935917_taxonomic_profile.tsv –samout SRR5935917_bowtie2.sam
–nproc 8 –no_map –tmp_dir main
* count_features.py –input metaphlan2_taxonomic_profiles.tsv –output
metaphlan2_species_counts_table.tsv –include s__ –filter t__ –reducesample-
name
* humann2_join_tables –input main –output metaphlan2_taxonomic_profiles.tsv
–file_name taxonomic_profile
* sample2markers.py –ifn_samples SRR5935917_bowtie2.sam –input_type
sam –output_dir strainphlan –nprocs 8 –min_read_depth 10
* strainphlan.py –ifn_samples *.markers –output_dir strainphlan –
print_clades_only > clades_list.txt
* extract_markers.py –mpa_pkl mpa_v20_m200.pkl –ifn_markers
all_markers.fasta –clade s__Bacteroides_vulgatus –ofn_markers
s__Bacteroides_vulgatus.markers.fasta
* strainphlan.py –ifn_samples *.markers –output_dir strainphlan –clades
s__Bacteroides_uniformis –nprocs_main 8 –keep_alignment_files
–ifn_markers s__Bacteroides_uniformis.markers.fasta > 0_clade.log

3.2.6.2 主要参数说明
 * –samout：输出比对SAM文件
 * –nproc：比对并行所用线程数量
 * –no_map：不存储--bowtie2out的比对文件，节省硬盘空间
 * –include：只统计包含某一字段的数量，比如这里只统计包含s__（也就是物种）字段的数量，也就是统计物种水平以及菌株种系水平的数量
 * –filter：不统计包含某一字段的数量，比如这里不统计包含t__（也就是菌株种系水平）字段的数量，结合–include s__参数，这里count_features.py只统计物种水平的数量
 * –reducesample-name：简化样本名称，只保留样本标记字段
 * –ifn_samples：输入每个样本比对文件或markers文件
 * –min_read_depth：比对上reads最少数量
 * –print_clades_only：只需给出菌株所属进化分支
 * –mpa_pkl：MetaPhlAn软件已构建的菌株标志蛋白的meta文件
 * –ifn_markers：输入MetaPhlAn软件已构建的菌株标志蛋白序列文件，或用户指定的某一特定菌株标志蛋白序列文件
 * –clade：选择某一特定进化分支分析所有样本的菌株分布
 * –ofn_markers：输出菌株标志蛋白序列
 * –keep_alignment_files：保留所有菌株标志序列的比对文件

3.2.6.3 物种分类数量统计表格

![**wmgx_report_figure9_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure9_1.png)

表格说明：filtered过滤条件是至少在10%样本中具有超过0.01%的物种分类丰度

3.2.6.4 基于物种水平丰度谱（前25种高丰度物种）和Bray-Curtis距离对样本进行PCoA降维分析

![**wmgx_report_figure10_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure10_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人，从PCoA图可以看出，健康人之间高丰度物种群落较为相似，而CD病人之间高丰度物种群落差距较大，这一结论与目前多篇文献结论一致。

3.2.6.5 基于属水平丰度谱（前25种高丰度属）和Bray-Curtis距离对样本进行PCoA降维分析

![**wmgx_report_figure12_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure12_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。比较基于物种水平和属水平丰度谱计算得到的样本间BC距离，可以发现，物种水平的菌群在CD病人和健康人之间具有较高分辨率。物种水平中，健康人之间BC距离比较紧密，与CD病人的差距更大。

3.2.6.6 基于物种水平丰度谱（前25种高丰度物种）对样本间Spearman相关性进行聚类分析

![**wmgx_report_figure14_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure14_1.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。

3.2.6.7 基于物种水平丰度谱（前25种高丰度物种）和Bray-Curtis距离对样本进行聚类分析

![**wmgx_report_figure15_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure15_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。比较对样本间Spearman相似性和BC距离的聚类分析结果，可以发现，这两种聚类分析结果一致。

3.2.6.8 基于属水平丰度谱（前25种高丰度属）对样本间Spearman相关性进行聚类分析

![**wmgx_report_figure16_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure16_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。

3.2.6.9 基于属水平丰度谱（前25种高丰度属）对样本间BC距离进行聚类分析

![**wmgx_report_figure17_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure17_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。比较对样本间Spearman相似性和BC距离的聚类分析结果，可以发现，这两种聚类分析结果一致。

3.2.6.10 各样本高丰度物种菌群组成图（前15种高丰度物种）

![**wmgx_report_figure19_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure19_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。从图中可以看出，健康人和CD病人在物种水平的菌群组成差异很大，健康人的菌群物种组成较为相似，除了样本SRR5950563，普氏杆菌Prevotella copri丰度在健康人样本中具有明显优势；而在CD病人的菌群物种组成变化较大，其中，普通拟杆菌Bacteroides vulgatus和单形拟杆菌Bacteroides uniformis是CD病人的优势菌群物种，而普通拟杆菌Bacteroides vulgatus是CD疾病的“明星物种”，文献报道它抑制了肠道黏液的分泌，导致CD疾病发生。

3.2.6.11 各样本高丰度属菌群组成图（前15种高丰度属）

![**wmgx_report_figure21_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure21_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。从图中可以看出，与物种水平组成相比，健康人和CD病人在属水平菌群组成差异相对较小，说明物种水平的菌群在CD病人和健康人之间具有较高分辨率，其中拟杆菌属Bacteroides是CD病人的优势属菌群，而普氏菌属Prevotella是健康人的优势属菌群。

In [ ]:
3.2.6.12 样本特异性菌株进化树

![**RAxML_bestTree.s__Eubacterium_rectale**](results/biobakery_workflows/strainphlan/RAxML_bestTree.s__Eubacterium_rectale.svg)

图说明：蓝色字体表示健康人样本，红色字体表示CD病人样本。

3.2.7 群落功能分析步骤

3.2.7.1 示例命令
* humann2 –input SRR5946998.fastq –output main –o-log SRR5946998.log
–threads 8 –taxonomic-profile SRR5946998_taxonomic_profile.tsv
* humann2_renorm_table –input SRR5935917_pathabundance.tsv –output
SRR5935917_pathabundance_relab.tsv –units relab –special n
* humann2_regroup_table –input SRR5935917_genefamilies.tsv –output
SRR5935917_ecs.tsv –groups uniref90_level4ec
* humann2_join_tables –input main –output genefamilies.tsv –file_name
genefamilies
* humann2_join_tables –input genes –output genefamilies_relab.tsv
* humann2_join_tables –input main –output pathabundance.tsv –file_name
pathabundance
* humann2_join_tables –input pathways –output pathabundance_relab.tsv
* humann2_join_tables –input regrouped –output ecs.tsv –file_name ecs
* humann2_join_tables –input ecs –output ecs_relab.tsv
* humann2_join_tables –input counts –output humann2_feature_counts.tsv
–file_name _relab_counts.tsv
* count_features.py –input pathabundance_relab.tsv –output humann2_
pathabundance_relab_counts.tsv –reduce-sample-name –ignoreun-
features –ignore-stratification
* count_features.py –input ecs_relab.tsv –output humann2_ecs_relab_counts.tsv
–reduce-sample-name –ignore-un-features –ignore-stratification
* count_features.py –input genefamilies_relab.tsv –output humann2_genefamilies_relab_counts.tsv
–reduce-sample-name –ignore-un-features –ignore-stratification

3.2.7.2 主要参数说明
* –taxonomic-profile：上游工具MetaPhlan产生的物种丰度谱
* –units：标准化算法，这里relab参数选项表示以相对丰度进行标准化
* –special：包含无特定功能，例如，UNMAPPED, UNINTEGRATED, and UNGROUPED
* –groups：功能群选项，这里以uniref90_level4ec单位，将基因组织成功能群
* –ignore-un-features：统计功能数量过程中不包括UNMAPPED, UNGROUPED, or UNINTEGRATED等无特定功能
* –ignore-stratification：统计功能数量过程中不按照物种分类水平进行分层统计，统计整体群落功能数量

3.2.7.3 基于高丰度通路对样本进行聚类分析（前50条高丰度通路，丰度值经过log10转化）

![**wmgx_report_figure26_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure26_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。从图中可以看出，CD病人样本SRR5936044的通路丰度与其它样本显著不同，通过查看meta表格可知，该病人服用过抗生素，这可能影响菌群功能。

3.2.7.4 基于高丰度通路对样本进行聚类分析（前50条高丰度通路，丰度值经过z-score转化）

![**wmgx_report_figure27_1_2**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure27_1_2.png)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。从图中可以看出，CD病人样本SRR5936044的通路丰度与其它样本显著不同。丰度值经过log10转化和z-score转化的结果相似。

3.2.7.5 所有样本高丰度通路统计表格（前50条高丰度通路）

![**wmgx_report_figure29_1**](results/biobakery_workflows_output_vis/figures/wmgx_report_figure29_1.png)

表格说明：Average表示该通路在所有样本丰度的平均值，Variance表示该通路在所有样本丰度的方差

#### 3.3 群落代谢物丰度谱构建

3.3.1 群落功能基因谱的数据准备

In [ ]:
genefamilies <- read.delim("/work/results/biobakery_workflows/humann2/merged/genefamilies_relab.tsv",stringsAsFactors=F,sep="\t",header=T)

In [ ]:
UniRef90 <- genefamilies[sapply(genefamilies$X..Gene.Family,function(x) length(strsplit(x,"\\|")[[1]])==1),]

In [ ]:
colnames(UniRef90) <- sapply(colnames(UniRef90),function(x) strsplit(x,"_")[[1]][1])

In [ ]:
colnames(UniRef90)[1]="ID"

In [ ]:
UniRef90_2 <- rbind(colnames(UniRef90),UniRef90)

In [ ]:
UniRef90_3 <- t(UniRef90_2)

In [ ]:
write.table(UniRef90_3,"/work/results/biobakery_workflows/humann2/merged/genefamilies_relab_input_for_MelonnPan.tsv",sep="\t",col.names=F,row.names=F,quote=F)

3.3.2 在服务器上安装MelonnPan软件，输入文件为3.3.1步骤处理好的genefamilies_relab_input_for_MelonnPan.tsv文件

3.3.3 服务器上运行命令：
Rscript predict_metabolites.R -i genefamilies_relab_input_for_MelonnPan.tsv -o melonnpan/

3.3.4 命令运行时间约3-4小时，运行完之后，从服务器下载melonnpan文件夹到本地/Volumes/Untitled/results

3.3.5 代谢物数量统计表格

In [1]:
MelonnPan_Predicted_Metabolites <- read.table("/work/results/melonnpan/MelonnPan_Predicted_Metabolites.txt",stringsAsFactors=F,sep="\t",header=T)

In [2]:
metabolites_number <- apply(MelonnPan_Predicted_Metabolites,1,function(x) sum(x[-1]>0))

In [3]:
filtered_index <- apply(MelonnPan_Predicted_Metabolites[,-1],2,function(x) sum(x>0.0001)>1)

In [4]:
MelonnPan_Predicted_Metabolites_filtered <- cbind(MelonnPan_Predicted_Metabolites[,1],MelonnPan_Predicted_Metabolites[,-1][,filtered_index])

In [5]:
metabolites_filtered_number <- apply(MelonnPan_Predicted_Metabolites_filtered,1,function(x) sum(x[-1]>0.0001))

In [6]:
Metabolites_data <- data.frame(Metabolites=metabolites_number,Metabolites_filtered=metabolites_filtered_number)

In [7]:
rownames(Metabolites_data) <- MelonnPan_Predicted_Metabolites$ID

In [8]:
write.table(Metabolites_data,"/work/results/melonnpan/metabolite_counts_table.txt",quote=F,sep="\t")

In [12]:
Metabolites_data

,Metabolites,Metabolites_filtered
,<int>,<int>
SRR5935846,80,16
SRR5935917,80,19
SRR5936044,80,19
SRR5936058,80,22
SRR5946729,80,18
SRR5946747,80,21
SRR5946795,80,18
SRR5946902,80,16
SRR5946998,80,17


3.3.6 基于代谢物丰度谱（前25种高丰度代谢物）和Bray-Curtis距离对样本进行PCoA降维分析

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/Metabolite_BC_PcoA.R \
    --metabolite_profile /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites.txt \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_plot /home/data/results/melonnpan/Top_25_metabolites_BC_PcoA.pdf'

Loading required package: permute
Loading required package: lattice
This is vegan 2.5-6
null device 
          1 


![**Top_25_metabolites_BC_PcoA**](results/melonnpan/Top_25_metabolites_BC_PcoA.pdf)

3.3.7 基于代谢物丰度谱（前25种高丰度代谢物）和Spearman相关性度量对样本进行聚类分析

In [9]:
MelonnPan_Predicted_Metabolites <- read.table("/work/results/melonnpan/MelonnPan_Predicted_Metabolites.txt",stringsAsFactors=F,sep="\t",header=T)

In [10]:
rownames(MelonnPan_Predicted_Metabolites) <- MelonnPan_Predicted_Metabolites$ID

In [11]:
MelonnPan_Predicted_Metabolites <- MelonnPan_Predicted_Metabolites[,-1]

In [12]:
write.table(t(MelonnPan_Predicted_Metabolites),"/work/results/melonnpan/MelonnPan_Predicted_Metabolites_input_for_hclust2.txt",quote=F,sep="\t")

In [4]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'hclust2.py -i /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites_input_for_hclust2.txt \
    -o /home/data/results/melonnpan/Metabolite_Spearman_Heatmap.pdf --ftop 25 \
    --f_dist_f spearman --s_dist_f spearman --cell_aspect_ratio 0.5 -l \
    --max_flabel_len 100 --max_slabel_len 100 --dpi 300 \
    --title "Top 25 metabolites by average abundance (Spearman)" \
    --title_fontsize 15'

/opt/conda/bin/hclust2.py:152: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col = self.args.sname_row if self.args.sname_row > -1 else None


![**Metabolite_Spearman_Heatmap**](results/melonnpan/Metabolite_Spearman_Heatmap.pdf)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。

3.3.8 基于代谢物丰度谱（前25种高丰度代谢物）和Bray-Curtis距离对样本进行聚类分析

In [5]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'hclust2.py -i /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites_input_for_hclust2.txt \
    -o /home/data/results/melonnpan/Metabolite_BrayCurtis_Heatmap.pdf --ftop 25 \
    --f_dist_f braycurtis --s_dist_f braycurtis --cell_aspect_ratio 0.5 -l \
    --max_flabel_len 100 --max_slabel_len 100 --dpi 300 \
    --title "Top 25 metabolites by average abundance (Bray-Curtis)" \
    --title_fontsize 15'

/opt/conda/bin/hclust2.py:152: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col = self.args.sname_row if self.args.sname_row > -1 else None


![**Metabolite_BrayCurtis_Heatmap**](results/melonnpan/Metabolite_BrayCurtis_Heatmap.pdf)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。

3.3.9 各样本高丰度代谢物组成图（前15种高丰度代谢物）

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/Metabolites_Barplot.R \
    --metabolite_profile /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites_input_for_hclust2.txt \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_plot /home/data/results/melonnpan/Top_15_metabolites_Barplot.pdf'

![**Top_15_metabolites_Barplot**](results/melonnpan/Top_15_metabolites_Barplot.pdf)

图说明：图中红色字体样本表示CD病人，蓝色字体样本表示健康人。

#### 3.4 群落结构、功能及代谢物的差异丰度分析和特征marker分析

创建存放差异分析结果文件夹

In [ ]:
!bash -c 'mkdir -p /work/results/metagenomeSeq/'

3.4.1 群落结构差异分析

3.4.1.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析群落结构的差异丰度变化

3.4.1.1.1 代码命令

In [ ]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_species_diagnosis.R \
    --taxonomic_profile /home/data/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --read_count /home/data/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_species.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA.txt'

参数说明：
* --taxonomic_profile：3.2.6步骤中MetaPhlan2产生的物种分类丰度谱
* --metadata：IBD项目的meta信息表格
* --read_count：每个样本经过质控之后，用于物种分类丰度谱构建的测序序列数量
* --output_all：输出所有物种的分析结果
* --output_DA：输出具有差异丰度变化物种的分析结果

从结果可以看出，一共有4个物种具有差异丰度变化，查看组间差异物种分类丰度谱/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA.txt

In [13]:
CD_vs_nonIBD_species_DA <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA.txt",stringsAsFactors=F,sep="\t",header=T)

In [14]:
CD_vs_nonIBD_species_DA

species,fold_change,adjustPvalue
<chr>,<dbl>,<dbl>
k__Bacteria|p__Verrucomicrobia|c__Verrucomicrobiae|o__Verrucomicrobiales|f__Verrucomicrobiaceae|g__Akkermansia|s__Akkermansia_muciniphila,0.0000,2.143816e-07
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_copri,0.0000,1.161919e-04
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_onderdonkii,0.0000,1.190214e-02
k__Bacteria|p__Proteobacteria|c__Deltaproteobacteria|o__Desulfovibrionales|f__Desulfovibrionaceae|g__Bilophila|s__Bilophila_wadsworthia,936.3356,1.790751e-02


3.4.1.1.2 基于差异物种丰度谱对样本进行聚类分析

In [1]:
DA <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA.txt",stringsAsFactors=F,sep="\t",header=T)

In [2]:
metaphlan2_taxonomic_profiles <- read.delim("/work/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv",stringsAsFactors=F,sep="\t",header=T)

In [3]:
species_taxonomic_profiles <- metaphlan2_taxonomic_profiles[sapply(metaphlan2_taxonomic_profiles$X.SampleID,function(x) length(strsplit(x,"\\|")[[1]])==7),]

In [4]:
species_taxonomic_profiles[,-1] <- species_taxonomic_profiles[,-1]/100

In [5]:
colnames(species_taxonomic_profiles)[-1] <- sapply(colnames(species_taxonomic_profiles)[-1],function(x) strsplit(x,"_")[[1]][1])

In [6]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [7]:
read_count <- read.delim("/work/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv",stringsAsFactor=F,sep="\t")

In [8]:
species_counts = sweep(species_taxonomic_profiles[,-1],2,read_count[match(colnames(species_taxonomic_profiles)[-1],read_count$X..samples),"total.reads"], "*")

In [9]:
species_counts = round(species_counts)

In [10]:
rownames(species_counts) <- species_taxonomic_profiles$X.SampleID

In [11]:
DA_profile <- species_counts[match(DA$species,rownames(species_counts)),]

In [15]:
write.table(DA_profile,"/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_profile.txt",quote=F,sep="\t")

In [16]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'hclust2.py -i /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_profile.txt \
    -o /home/data/results/metagenomeSeq/DA_species_Heatmap.pdf \
    --cell_aspect_ratio 1.5 -l \
    --max_flabel_len 100 --max_slabel_len 100 --dpi 300 \
    --title "Species with differential abundance" \
    --title_fontsize 15 \
    --colorbar_font_size 10'

/opt/conda/bin/hclust2.py:152: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col = self.args.sname_row if self.args.sname_row > -1 else None


![**DA_species_Heatmap**](results/metagenomeSeq/DA_species_Heatmap.pdf)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。在4个具有差异物种的丰度谱中，CD病人聚成一支。差异物种Akkermansia_muciniphila、Alistipes_onderdonkii和Prevotella_copri显著下调，而Bilophila_wadsworthia显著上调。之前研究表明Bilophila_wadsworthia细菌肠道炎症失调的患者中高水平繁殖。

3.4.1.2 以抗生素使用情况作为混杂因素，在CD疾病和健康两种状态下，分析群落结构的差异丰度变化

说明：在样本数量足够多时，理论上，可以同时考虑多个混杂因素，考虑到测试数据样本量较少，因此我们这里混杂因素分析每次只考虑一个混杂因素。

3.4.1.2.1 代码命令

In [81]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_species_diagnosis_confounder.R \
    --taxonomic_profile /home/data/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --read_count /home/data/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_confounder_Antibiotics.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_Antibiotics.txt \
    --confounder Antibiotics'    

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


参数说明：
* --taxonomic_profile：3.2.6步骤中MetaPhlan2产生的物种分类丰度谱
* --metadata：IBD项目的meta信息表格
* --read_count：每个样本经过质控之后，用于物种分类丰度谱构建的测序序列数量
* --output_all：输出所有物种的分析结果
* --output_DA：输出具有差异丰度变化物种的分析结果
* --confounder：混杂因素名称

从结果可以看出，在差异分析模型中考虑了抗生素使用情况这个混杂因素之后，一共有74个物种具有差异丰度变化，查看组间差异物种分类丰度谱/home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_Antibiotics.txt

In [2]:
CD_vs_nonIBD_species_DA_confounder_Antibiotics <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_Antibiotics.txt",stringsAsFactors=F,sep="\t",header=T)

In [3]:
CD_vs_nonIBD_species_DA_confounder_Antibiotics

species,fold_change,adjustPvalue
<chr>,<dbl>,<dbl>
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_copri,0.000000e+00,3.819061e-12
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Coprococcus|s__Coprococcus_sp_ART55_1,0.000000e+00,3.725740e-07
k__Bacteria|p__Verrucomicrobia|c__Verrucomicrobiae|o__Verrucomicrobiales|f__Verrucomicrobiaceae|g__Akkermansia|s__Akkermansia_muciniphila,0.000000e+00,8.297757e-07
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_sp_1_1_6,Inf,1.733825e-06
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Oscillospiraceae|g__Oscillibacter|s__Oscillibacter_sp_KLE_1745,Inf,1.733825e-06
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_sp_4_3_47FAA,0.000000e+00,2.180688e-06
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_symbiosum,Inf,2.545477e-06
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_clostridioforme,Inf,4.075885e-06
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_coprocola,0.000000e+00,4.627589e-06


3.4.1.2.2 基于差异物种丰度谱对样本进行聚类分析

In [5]:
DA <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_Antibiotics.txt",stringsAsFactors=F,sep="\t",header=T)

In [6]:
metaphlan2_taxonomic_profiles <- read.delim("/work/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv",stringsAsFactors=F,sep="\t",header=T)

In [7]:
species_taxonomic_profiles <- metaphlan2_taxonomic_profiles[sapply(metaphlan2_taxonomic_profiles$X.SampleID,function(x) length(strsplit(x,"\\|")[[1]])==7),]

In [8]:
species_taxonomic_profiles[,-1] <- species_taxonomic_profiles[,-1]/100

In [9]:
colnames(species_taxonomic_profiles)[-1] <- sapply(colnames(species_taxonomic_profiles)[-1],function(x) strsplit(x,"_")[[1]][1])

In [11]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [12]:
read_count <- read.delim("/work/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv",stringsAsFactor=F,sep="\t")

In [13]:
species_counts = sweep(species_taxonomic_profiles[,-1],2,read_count[match(colnames(species_taxonomic_profiles)[-1],read_count$X..samples),"total.reads"], "*")

In [14]:
species_counts = round(species_counts)

In [15]:
rownames(species_counts) <- species_taxonomic_profiles$X.SampleID

In [16]:
DA_profile <- species_counts[match(DA$species,rownames(species_counts)),]

In [17]:
write.table(DA_profile,"/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_Antibiotics_profile.txt",quote=F,sep="\t")

In [10]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'hclust2.py -i /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_Antibiotics_profile.txt \
    -o /home/data/results/metagenomeSeq/DA_species_confounder_Antibiotics_BrayCurtis_Heatmap.pdf \
    --cell_aspect_ratio 0.5 -l \
    --max_flabel_len 100 --max_slabel_len 100 --dpi 300 \
    --title "Species with differential abundance (Bray-Curtis)" \
    --title_fontsize 8 --f_dist_f braycurtis --s_dist_f braycurtis \
    --colorbar_font_size 6 \
    --flabel_size 4 \
    --slabel_size 4'

/opt/conda/bin/hclust2.py:152: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col = self.args.sname_row if self.args.sname_row > -1 else None


![**DA_species_confounder_Antibiotics_BrayCurtis_Heatmap**](results/metagenomeSeq/DA_species_confounder_Antibiotics_BrayCurtis_Heatmap.pdf)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。

3.4.1.3 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析群落结构的差异丰度变化

3.4.1.3.1 代码命令

In [82]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_species_diagnosis_confounder.R \
    --taxonomic_profile /home/data/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --read_count /home/data/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_confounder_consent_age.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_consent_age.txt \
    --confounder consent_age'    

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


参数说明：
* --taxonomic_profile：3.2.6步骤中MetaPhlan2产生的物种分类丰度谱
* --metadata：IBD项目的meta信息表格
* --read_count：每个样本经过质控之后，用于物种分类丰度谱构建的测序序列数量
* --output_all：输出所有物种的分析结果
* --output_DA：输出具有差异丰度变化物种的分析结果
* --confounder：混杂因素名称

从结果可以看出，在差异分析模型中考虑了年龄混杂因素之后，一共有40个物种具有差异丰度变化，查看组间差异物种分类丰度谱/home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_consent_age.txt

In [2]:
CD_vs_nonIBD_species_DA_confounder_consent_age <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_consent_age.txt",stringsAsFactors=F,sep="\t",header=T)

In [4]:
CD_vs_nonIBD_species_DA_confounder_consent_age

species,fold_change,adjustPvalue
<chr>,<dbl>,<dbl>
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_copri,0.000000e+00,3.550525e-09
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_clostridioforme,Inf,6.172808e-05
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_onderdonkii,0.000000e+00,6.172808e-05
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Oscillospiraceae|g__Oscillibacter|s__Oscillibacter_sp_KLE_1745,Inf,1.055650e-04
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_symbiosum,Inf,4.264747e-04
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_sp_3_1_31,Inf,4.412852e-04
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_asparagiforme,Inf,5.264061e-04
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_sp_KLE_1755,Inf,1.366855e-03
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Clostridiaceae|g__Clostridium|s__Clostridium_nexile,Inf,1.366855e-03


3.4.1.3.2 基于差异物种丰度谱对样本进行聚类分析

In [5]:
DA <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_consent_age.txt",stringsAsFactors=F,sep="\t",header=T)

In [6]:
metaphlan2_taxonomic_profiles <- read.delim("/work/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv",stringsAsFactors=F,sep="\t",header=T)

In [7]:
species_taxonomic_profiles <- metaphlan2_taxonomic_profiles[sapply(metaphlan2_taxonomic_profiles$X.SampleID,function(x) length(strsplit(x,"\\|")[[1]])==7),]

In [8]:
species_taxonomic_profiles[,-1] <- species_taxonomic_profiles[,-1]/100

In [9]:
colnames(species_taxonomic_profiles)[-1] <- sapply(colnames(species_taxonomic_profiles)[-1],function(x) strsplit(x,"_")[[1]][1])

In [10]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [11]:
read_count <- read.delim("/work/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv",stringsAsFactor=F,sep="\t")

In [12]:
species_counts = sweep(species_taxonomic_profiles[,-1],2,read_count[match(colnames(species_taxonomic_profiles)[-1],read_count$X..samples),"total.reads"], "*")

In [13]:
species_counts = round(species_counts)

In [14]:
rownames(species_counts) <- species_taxonomic_profiles$X.SampleID

In [15]:
DA_profile <- species_counts[match(DA$species,rownames(species_counts)),]

In [16]:
write.table(DA_profile,"/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_consent_age_profile.txt",quote=F,sep="\t")

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'hclust2.py -i /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_consent_age_profile.txt \
    -o /home/data/results/metagenomeSeq/DA_species_confounder_consent_age_BrayCurtis_Heatmap.pdf \
    --cell_aspect_ratio 0.5 -l \
    --max_flabel_len 100 --max_slabel_len 100 --dpi 300 \
    --title "Species with differential abundance (Bray-Curtis)" \
    --title_fontsize 8 --f_dist_f braycurtis --s_dist_f braycurtis \
    --colorbar_font_size 6 \
    --flabel_size 4 \
    --slabel_size 4'

/opt/conda/bin/hclust2.py:152: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col = self.args.sname_row if self.args.sname_row > -1 else None


![**DA_species_confounder_consent_age_BrayCurtis_Heatmap**](results/metagenomeSeq/DA_species_confounder_consent_age_BrayCurtis_Heatmap.pdf)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。从结果可以看出，考虑了年龄这个混杂因素之后，健康人样本聚成了一类。

3.4.1.4 以性别作为混杂因素，在CD疾病和健康两种状态下，分析群落结构的差异丰度变化

3.4.1.4.1 代码命令

In [83]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_species_diagnosis_confounder.R \
    --taxonomic_profile /home/data/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --read_count /home/data/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_confounder_gender.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_gender.txt \
    --confounder gender'    

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


参数说明：
* --taxonomic_profile：3.2.6步骤中MetaPhlan2产生的物种分类丰度谱
* --metadata：IBD项目的meta信息表格
* --read_count：每个样本经过质控之后，用于物种分类丰度谱构建的测序序列数量
* --output_all：输出所有物种的分析结果
* --output_DA：输出具有差异丰度变化物种的分析结果
* --confounder：混杂因素名称

从结果可以看出，在差异分析模型中考虑了性别混杂因素之后，一共有29个物种具有差异丰度变化，查看组间差异物种分类丰度谱/home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_gender.txt

In [2]:
CD_vs_nonIBD_species_DA_confounder_gender <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_gender.txt",stringsAsFactors=F,sep="\t",header=T)

In [3]:
CD_vs_nonIBD_species_DA_confounder_gender

species,fold_change,adjustPvalue
<chr>,<dbl>,<dbl>
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Prevotellaceae|g__Prevotella|s__Prevotella_copri,0.000000e+00,5.339587e-11
k__Bacteria|p__Verrucomicrobia|c__Verrucomicrobiae|o__Verrucomicrobiales|f__Verrucomicrobiaceae|g__Akkermansia|s__Akkermansia_muciniphila,0.000000e+00,4.512651e-06
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_onderdonkii,0.000000e+00,6.559316e-05
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Lachnospiraceae|g__Coprococcus|s__Coprococcus_sp_ART55_1,0.000000e+00,1.724816e-04
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Ruminococcaceae|g__Ruminococcus|s__Ruminococcus_callidus,0.000000e+00,1.724816e-04
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_coprocola,0.000000e+00,1.968504e-04
k__Bacteria|p__Firmicutes|c__Clostridia|o__Clostridiales|f__Eubacteriaceae|g__Eubacterium|s__Eubacterium_siraeum,6.097287e-02,3.134697e-04
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Rikenellaceae|g__Alistipes|s__Alistipes_finegoldii,0.000000e+00,5.068566e-04
k__Bacteria|p__Bacteroidetes|c__Bacteroidia|o__Bacteroidales|f__Bacteroidaceae|g__Bacteroides|s__Bacteroides_sp_1_1_6,Inf,5.536658e-04


3.4.1.4.2 基于差异物种丰度谱对样本进行聚类分析

In [3]:
DA <- read.table("/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_gender.txt",stringsAsFactors=F,sep="\t",header=T)

In [4]:
metaphlan2_taxonomic_profiles <- read.delim("/work/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv",stringsAsFactors=F,sep="\t",header=T)

In [5]:
species_taxonomic_profiles <- metaphlan2_taxonomic_profiles[sapply(metaphlan2_taxonomic_profiles$X.SampleID,function(x) length(strsplit(x,"\\|")[[1]])==7),]

In [6]:
species_taxonomic_profiles[,-1] <- species_taxonomic_profiles[,-1]/100

In [7]:
colnames(species_taxonomic_profiles)[-1] <- sapply(colnames(species_taxonomic_profiles)[-1],function(x) strsplit(x,"_")[[1]][1])

In [8]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [9]:
read_count <- read.delim("/work/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv",stringsAsFactor=F,sep="\t")

In [10]:
species_counts = sweep(species_taxonomic_profiles[,-1],2,read_count[match(colnames(species_taxonomic_profiles)[-1],read_count$X..samples),"total.reads"], "*")

In [11]:
species_counts = round(species_counts)

In [12]:
rownames(species_counts) <- species_taxonomic_profiles$X.SampleID

In [13]:
DA_profile <- species_counts[match(DA$species,rownames(species_counts)),]

In [14]:
write.table(DA_profile,"/work/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_gender_profile.txt",quote=F,sep="\t")

In [2]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'hclust2.py -i /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_confounder_gender_profile.txt \
    -o /home/data/results/metagenomeSeq/DA_species_confounder_gender_BrayCurtis_Heatmap.pdf \
    --cell_aspect_ratio 0.5 -l \
    --max_flabel_len 100 --max_slabel_len 100 --dpi 300 \
    --title "Species with differential abundance (Bray-Curtis)" \
    --title_fontsize 8 --f_dist_f braycurtis --s_dist_f braycurtis \
    --colorbar_font_size 6 \
    --flabel_size 4 \
    --slabel_size 4'

/opt/conda/bin/hclust2.py:152: FutureWarning: read_table is deprecated, use read_csv instead.
  index_col = self.args.sname_row if self.args.sname_row > -1 else None


![**DA_species_confounder_gender_BrayCurtis_Heatmap**](results/metagenomeSeq/DA_species_confounder_gender_BrayCurtis_Heatmap.pdf)

图说明：图中亮蓝色方框表示健康人样本，其余样本表示CD病人。从结果可以看出，考虑了性别这个混杂因素之后，健康人样本聚成了一类。

3.4.1.5 尝试去除使用抗生素样本这一混杂因素，分析群落结构的差异丰度变化，探索混杂因素对差异分析的影响

In [77]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_diagnosis_remove_use_Antibiotics.R \
    --taxonomic_profile /home/data/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --read_count /home/data/results/biobakery_workflows_output_vis/data/humann2_read_and_species_count_table.tsv \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_remove_use_Antibiotics.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_species_DA_remove_use_Antibiotics.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.2 群落结构特征marker分析

3.4.2.1 输入文件准备

In [1]:
metaphlan2_taxonomic_profiles <- read.delim("/work/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
species_taxonomic_profiles <- metaphlan2_taxonomic_profiles[sapply(metaphlan2_taxonomic_profiles$X.SampleID,function(x) length(strsplit(x,"\\|")[[1]])==7),]

In [3]:
colnames(species_taxonomic_profiles)[-1] <- sapply(colnames(species_taxonomic_profiles)[-1],function(x) strsplit(x,"_")[[1]][1])

In [4]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [5]:
species_taxonomic_profiles_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(species_taxonomic_profiles)[-1],IBD_test_metadata$Run),"diagnosis"]),species_taxonomic_profiles)

In [6]:
write.table(species_taxonomic_profiles_2,"/work/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.2.2 命令

说明：特征marker分析步骤运行时间大概25分钟，输出结果文件夹放在/home/data/results/metaml/species_profiles_lasso_rf*文件。

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'python /opt/metaml/classification.py /home/data/results/biobakery_workflows/metaphlan2/merged/metaphlan2_taxonomic_profiles_input_for_MetAML.tsv \
/home/data/results/metaml/species_profiles_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w'

/opt/conda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:27: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:412: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/opt/metaml/classification.py:49: RuntimeWarning: invalid value encountered in divide
  t = abs(el.coef_)/sum(abs(el.coef_))
/opt/conda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: Precision is ill-de

3.4.3 群落功能差异分析——整体群落通路水平

3.4.3.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析整体群落通路的差异丰度变化

In [2]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_path_diagnosis.R \
    --path_profile /home/data/results/biobakery_workflows/humann2/merged/pathabundance.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_path.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_path_DA.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.3.2 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析整体群落通路的差异丰度变化

In [89]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_path_diagnosis_confounder.R \
    --path_profile /home/data/results/biobakery_workflows/humann2/merged/pathabundance.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_path_confounder_consent_age.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_path_DA_confounder_consent_age.txt \
    --confounder consent_age'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.4 群落功能差异分析——物种个体通路水平

3.4.4.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析物种个体通路的差异丰度变化

In [13]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_stratified_path_diagnosis.R \
    --path_profile /home/data/results/biobakery_workflows/humann2/merged/pathabundance.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_path_stratified.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_path_DA_stratified.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.4.2 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析物种个体通路的差异丰度变化

In [10]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_stratified_path_diagnosis_confounder.R \
    --path_profile /home/data/results/biobakery_workflows/humann2/merged/pathabundance.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_path_stratified_confounder_consent_age.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_path_DA_stratified_confounder_consent_age.txt \
    --confounder consent_age'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.5 群落功能差异分析——整体群落ECS蛋白酶家族水平

3.4.5.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析整体群落ECS蛋白酶家族的差异丰度变化

In [98]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_ecs_diagnosis.R \
    --ecs_profile /home/data/results/biobakery_workflows/humann2/merged/ecs.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_ecs.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_ecs_DA.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.5.2 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析整体群落ECS蛋白酶家族的差异丰度变化

In [101]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_ecs_diagnosis_confounder.R \
    --ecs_profile /home/data/results/biobakery_workflows/humann2/merged/ecs.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_ecs_confounder_consent_age.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_ecs_DA_confounder_consent_age.txt \
    --confounder consent_age'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.6 群落功能差异分析——物种个体ECS蛋白酶家族水平

3.4.6.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析物种个体ECS蛋白酶家族的差异丰度变化

In [104]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_stratified_ecs_diagnosis.R \
    --ecs_profile /home/data/results/biobakery_workflows/humann2/merged/ecs.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_ecs.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_ecs_DA.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.6.2 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析物种个体ECS蛋白酶家族的差异丰度变化

In [106]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_stratified_ecs_diagnosis_confounder.R \
    --ecs_profile /home/data/results/biobakery_workflows/humann2/merged/ecs.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_ecs_confounder_consent_age.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_ecs_DA_confounder_consent_age.txt \
    --confounder consent_age'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.7 群落功能差异分析——整体群落UniRef90基因家族水平

3.4.7.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析整体群落UniRef90基因家族的差异丰度变化

In [2]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_genefamilies_diagnosis.R \
    --genefamilies_profile /home/data/results/biobakery_workflows/humann2/merged/genefamilies.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_genefamilies.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_genefamilies_DA.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.7.2 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析整体群落UniRef90基因家族的差异丰度变化

In [3]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_genefamilies_diagnosis_confounder.R \
    --genefamilies_profile /home/data/results/biobakery_workflows/humann2/merged/genefamilies.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_genefamilies_confounder_Antibiotics.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_genefamilies_DA_confounder_Antibiotics.txt \
    --confounder Antibiotics'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.8 群落功能差异分析——物种个体UniRef90基因家族水平

3.4.8.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析物种个体UniRef90基因家族的差异丰度变化

In [2]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_stratified_genefamilies_diagnosis.R \
    --genefamilies_profile /home/data/results/biobakery_workflows/humann2/merged/genefamilies.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_genefamilies.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_genefamilies_DA.txt'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.8.2 以年龄作为混杂因素，在CD疾病和健康两种状态下，分析物种个体UniRef90基因家族的差异丰度变化

In [5]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/metagenomeSeq_stratified_genefamilies_diagnosis_confounder.R \
    --genefamilies_profile /home/data/results/biobakery_workflows/humann2/merged/genefamilies.tsv \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_genefamilies_confounder_consent_age.txt \
    --output_DA /home/data/results/metagenomeSeq/CD_vs_nonIBD_stratified_genefamilies_DA_confounder_consent_age.txt \
    --confounder consent_age'

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Welcome to Bioconductor

    Vignettes contain introductory material; view with


3.4.9 群落功能特征marker分析——整体群落通路水平

3.4.9.1 输入文件准备

In [1]:
pathabundance <- read.delim("/work/results/biobakery_workflows/humann2/merged/pathabundance.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
pathabundance <- pathabundance[sapply(pathabundance$X..Pathway,function(x) !(strsplit(x,"\\|")[[1]][1] %in% c("UNMAPPED","UNINTEGRATED")) & length(strsplit(x,"\\|")[[1]])==1),]

In [3]:
colnames(pathabundance)[-1] <- sapply(colnames(pathabundance)[-1],function(x) strsplit(x,"_")[[1]][1])

In [4]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [5]:
pathabundance_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(pathabundance)[-1],IBD_test_metadata$Run),"diagnosis"]),pathabundance)

In [6]:
pathabundance_2$X..Pathway[-1] <- paste("pathway",pathabundance_2$X..Pathway[-1],sep="_")

In [7]:
write.table(pathabundance_2,"/work/results/biobakery_workflows/humann2/merged/pathabundance_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.9.2 命令

In [3]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'python /opt/metaml/classification.py /home/data/results/biobakery_workflows/humann2/merged/pathabundance_input_for_MetAML.tsv \
/home/data/results/metaml/pathway_abundance_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z pathway_'

/opt/conda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:27: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:412: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/opt/metaml/classification.py:49: RuntimeWarning: invalid value encountered in divide
  t = abs(el.coef_)/sum(abs(el.coef_))
/opt/conda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:490: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/opt/conda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being s

3.4.10 群落功能特征marker分析——物种个体通路水平

3.4.10.1 输入文件准备

In [ ]:
pathabundance <- read.delim("/work/results/biobakery_workflows/humann2/merged/pathabundance.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
pathabundance <- pathabundance[sapply(pathabundance$X..Pathway,function(x) !(strsplit(x,"\\|")[[1]][1] %in% c("UNMAPPED","UNINTEGRATED")) & length(strsplit(x,"\\|")[[1]])==2),]

In [3]:
pathabundance <- pathabundance[sapply(pathabundance$X..Pathway,function(x) length(strsplit(strsplit(x,"\\|")[[1]][2],"s__")[[1]])==2),]

In [4]:
colnames(pathabundance)[-1] <- sapply(colnames(pathabundance)[-1],function(x) strsplit(x,"_")[[1]][1])

In [5]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [6]:
pathabundance_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(pathabundance)[-1],IBD_test_metadata$Run),"diagnosis"]),pathabundance)

In [7]:
pathabundance_2$X..Pathway[-1] <- paste("pathway",pathabundance_2$X..Pathway[-1],sep="_")

In [8]:
write.table(pathabundance_2,"/work/results/biobakery_workflows/humann2/merged/pathabundance_stratified_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.10.2 命令

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'python /opt/metaml/classification.py /home/data/results/biobakery_workflows/humann2/merged/pathabundance_stratified_input_for_MetAML.tsv \
/home/data/results/metaml/pathway_abundance_stratified_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z pathway_'

/opt/conda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:27: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:412: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/opt/metaml/classification.py:49: RuntimeWarning: invalid value encountered in divide
  t = abs(el.coef_)/sum(abs(el.coef_))
/opt/conda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:490: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/opt/conda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being s

3.4.11 群落功能特征marker分析——整体群落ECS蛋白酶家族水平

3.4.11.1 输入文件准备

In [1]:
ecsabundance <- read.delim("/work/results/biobakery_workflows/humann2/merged/ecs.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
ecsabundance <- ecsabundance[sapply(ecsabundance$X..Gene.Family,function(x) !(strsplit(x,"\\|")[[1]][1] %in% c("UNMAPPED","UNGROUPED")) & length(strsplit(x,"\\|")[[1]])==1),]

In [3]:
colnames(ecsabundance)[-1] <- sapply(colnames(ecsabundance)[-1],function(x) strsplit(x,"_")[[1]][1])

In [4]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [5]:
ecsabundance_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(ecsabundance)[-1],IBD_test_metadata$Run),"diagnosis"]),ecsabundance)

In [6]:
ecsabundance_2$X..Gene.Family[-1] <- paste("ecs",ecsabundance_2$X..Gene.Family[-1],sep="_")

In [7]:
write.table(ecsabundance_2,"/work/results/biobakery_workflows/humann2/merged/ecsabundance_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.11.2 命令

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'python /opt/metaml/classification.py /home/data/results/biobakery_workflows/humann2/merged/ecsabundance_input_for_MetAML.tsv \
/home/data/results/metaml/ecsabundance_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z ecs_'

/opt/conda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:27: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:412: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/opt/metaml/classification.py:49: RuntimeWarning: invalid value encountered in divide
  t = abs(el.coef_)/sum(abs(el.coef_))
/opt/conda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:490: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/opt/conda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/

3.4.12 群落功能特征marker分析——物种个体ECS蛋白酶家族水平

3.4.12.1 输入文件准备

In [1]:
ecsabundance <- read.delim("/work/results/biobakery_workflows/humann2/merged/ecs.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
ecsabundance <- ecsabundance[sapply(ecsabundance$X..Gene.Family,function(x) !(strsplit(x,"\\|")[[1]][1] %in% c("UNMAPPED","UNGROUPED")) & length(strsplit(x,"\\|")[[1]])==2),]

In [3]:
ecsabundance <- ecsabundance[sapply(ecsabundance$X..Gene.Family,function(x) length(strsplit(strsplit(x,"\\|")[[1]][2],"s__")[[1]])==2),]

In [4]:
colnames(ecsabundance)[-1] <- sapply(colnames(ecsabundance)[-1],function(x) strsplit(x,"_")[[1]][1])

In [5]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [6]:
ecsabundance_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(ecsabundance)[-1],IBD_test_metadata$Run),"diagnosis"]),ecsabundance)

In [7]:
ecsabundance_2$X..Gene.Family[-1] <- paste("ecs",ecsabundance_2$X..Gene.Family[-1],sep="_")

In [8]:
write.table(ecsabundance_2,"/work/results/biobakery_workflows/humann2/merged/ecsabundance_stratified_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.12.2 命令

In [1]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'python /opt/metaml/classification.py /home/data/results/biobakery_workflows/humann2/merged/ecsabundance_stratified_input_for_MetAML.tsv \
/home/data/results/metaml/ecsabundance_stratified_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z ecs_'

/opt/conda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:27: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:412: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/opt/conda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:490: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/opt/conda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars


3.4.13 群落功能特征marker分析——整体群落UniRef90基因家族水平

3.4.13.1 输入文件准备

In [1]:
genefamiliesabundance <- read.delim("/work/results/biobakery_workflows/humann2/merged/genefamilies.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
genefamiliesabundance <- genefamiliesabundance[sapply(genefamiliesabundance$X..Gene.Family,function(x) !(strsplit(x,"\\|")[[1]][1] %in% c("UNMAPPED","UniRef90_unknown")) & length(strsplit(x,"\\|")[[1]])==1),]

In [3]:
colnames(genefamiliesabundance)[-1] <- sapply(colnames(genefamiliesabundance)[-1],function(x) strsplit(x,"_")[[1]][1])

In [4]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [5]:
genefamiliesabundance_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(genefamiliesabundance)[-1],IBD_test_metadata$Run),"diagnosis"]),genefamiliesabundance)

In [6]:
genefamiliesabundance_2$X..Gene.Family[-1] <- paste("genefamilies",genefamiliesabundance_2$X..Gene.Family[-1],sep="_")

In [7]:
write.table(genefamiliesabundance_2,"/work/results/biobakery_workflows/humann2/merged/genefamiliesabundance_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.13.2 命令（由于该命令运行时间较长，转移到服务器上运行）

python classification.py genefamiliesabundance_input_for_MetAML.tsv genefamiliesabundance_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z genefamilies_'

3.4.14 群落功能特征marker分析——物种个体UniRef90基因家族水平

3.4.14.1 输入文件准备

In [1]:
genefamiliesabundance <- read.delim("/work/results/biobakery_workflows/humann2/merged/genefamilies.tsv",stringsAsFactors=F,sep="\t",header=T)

In [2]:
genefamiliesabundance <- genefamiliesabundance[sapply(genefamiliesabundance$X..Gene.Family,function(x) !(strsplit(x,"\\|")[[1]][1] %in% c("UNMAPPED","UniRef90_unknown")) & length(strsplit(x,"\\|")[[1]])==2),]

In [3]:
genefamiliesabundance <- genefamiliesabundance[sapply(genefamiliesabundance$X..Gene.Family,function(x) length(strsplit(strsplit(x,"\\|")[[1]][2],"s__")[[1]])==2),]

In [4]:
colnames(genefamiliesabundance)[-1] <- sapply(colnames(genefamiliesabundance)[-1],function(x) strsplit(x,"_")[[1]][1])

In [5]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [6]:
genefamiliesabundance_2 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(genefamiliesabundance)[-1],IBD_test_metadata$Run),"diagnosis"]),genefamiliesabundance)

In [7]:
genefamiliesabundance_2$X..Gene.Family[-1] <- paste("genefamilies",genefamiliesabundance_2$X..Gene.Family[-1],sep="_")

In [8]:
write.table(genefamiliesabundance_2,"/work/results/biobakery_workflows/humann2/merged/genefamiliesabundance_stratified_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.14.2 命令（由于该命令运行时间较长，转移到服务器上运行）

python classification.py genefamiliesabundance_stratified_input_for_MetAML.tsv genefamiliesabundance_stratified_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z genefamilies_'

3.4.15 群落代谢物差异分析 (整体群落)

3.4.15.1 不考虑混杂因素，在CD疾病和健康两种状态下，分析群落代谢物的差异丰度变化

In [12]:
!bash -c 'mkdir -p /work/results/wilcox/'

In [13]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/nlme_metabolites.R \
    --metabolites_profile /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites.txt \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/wilcox/CD_vs_nonIBD_metabolites.txt \
    --output_DA /home/data/results/wilcox/CD_vs_nonIBD_metabolites_DA.txt'

Warning messages:
1: In wilcox.test.default(as.numeric(x[subset(IBD_test_metadata, diagnosis ==  :
  cannot compute exact p-value with ties
2: In wilcox.test.default(as.numeric(x[subset(IBD_test_metadata, diagnosis ==  :
  cannot compute exact p-value with ties
[1] "Number of metabolites with differential abundance:0"


3.4.15.2 同时以抗生素使用情况和年龄作为混杂因素，在CD疾病和健康两种状态下，分析群落代谢物的差异丰度变化

In [14]:
!bash -c 'mkdir -p /work/results/nlme/'

In [15]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'Rscript /home/data/nlme_metabolites_confounder.R \
    --metabolites_profile /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites.txt \
    --metadata /home/data/IBD_test_metadata.txt \
    --output_all /home/data/results/nlme/CD_vs_nonIBD_metabolites_confounder.txt \
    --output_DA /home/data/results/nlme/CD_vs_nonIBD_metabolites_DA_confounder.txt'

[1] "Number of metabolites with differential abundance:3"


3.4.16 群落代谢物特征marker分析 (整体群落)

3.4.16.1 输入文件准备

In [1]:
MelonnPan_Predicted_Metabolites <- read.table("/work/results/melonnpan/MelonnPan_Predicted_Metabolites.txt",stringsAsFactors=F,sep="\t",header=T)

In [2]:
rownames(MelonnPan_Predicted_Metabolites) <- MelonnPan_Predicted_Metabolites$ID 

In [3]:
MelonnPan_Predicted_Metabolites <- MelonnPan_Predicted_Metabolites[,-1]

In [4]:
MelonnPan_Predicted_Metabolites_2 <- t(MelonnPan_Predicted_Metabolites)

In [5]:
MelonnPan_Predicted_Metabolites_3 <- data.frame(metabolites=rownames(MelonnPan_Predicted_Metabolites_2),MelonnPan_Predicted_Metabolites_2)

In [6]:
IBD_test_metadata <- read.table("/work/IBD_test_metadata.txt",stringsAsFactors=F,sep="\t",header=T)

In [7]:
MelonnPan_Predicted_Metabolites_3$metabolites <- as.character(MelonnPan_Predicted_Metabolites_3$metabolites)

In [8]:
MelonnPan_Predicted_Metabolites_4 <- rbind(c("diagnosis",IBD_test_metadata[match(colnames(MelonnPan_Predicted_Metabolites_3)[-1],IBD_test_metadata$Run),"diagnosis"]),MelonnPan_Predicted_Metabolites_3)

In [9]:
MelonnPan_Predicted_Metabolites_4$metabolites[-1] <- paste("metabolites",MelonnPan_Predicted_Metabolites_4$metabolites[-1],sep="_")

In [10]:
write.table(MelonnPan_Predicted_Metabolites_4,"/work/results/melonnpan/MelonnPan_Predicted_Metabolites_input_for_MetAML.tsv",quote=F,sep="\t",row.names=F,col.names=F)

3.4.16.2 命令

In [8]:
!sudo docker run --rm -v /Volumes/Untitled:/home/data biobakery-workflows-add-r-6 bash \
    -c 'python /opt/metaml/classification.py /home/data/results/melonnpan/MelonnPan_Predicted_Metabolites_input_for_MetAML.tsv \
/home/data/results/metaml/metabolites_lasso_rf -d 1:diagnosis:CD -i lasso -l rf -w -z metabolites_'

/opt/conda/lib/python2.7/site-packages/sklearn/ensemble/weight_boosting.py:27: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/opt/conda/lib/python2.7/site-packages/sklearn/cross_validation.py:412: Warning: The least populated class in y has only 5 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=10.
  % (min_labels, self.n_folds)), Warning)
/opt/conda/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:490: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)
/opt/conda/lib/python2.7/site-packages/sklearn/metrics/metrics.py:1771: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python2.7/site-p